# Acquisition Models for SPECT
This demonstration shows how to set-up and use SIRF acquisition models for SPECT. You should have tried the `introduction` notebook first. The current notebook briefly repeats some items without explanation.

This demo is a jupyter notebook, i.e. intended to be run step by step.
You could export it as a Python file and run it one go, but that might
make little sense as the figures are not labelled.

Forward projection demo: creates an image, projects it to simulate
acquisition data and backprojects



Authors: Daniel Deidda, Sam Porter, Kris Thielemans

First version: 13th of May 2022 

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2022 National Physical Laboratory.  
Copyright 2015 - 2019, 2022 University College London.  

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In [ ]:
# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
from sirf.Utilities import examples_data_path
from sirf.STIR import show_2D_array
import sirf
# Initial imports etc
import numpy
import os
import sys
import shutil
import brainweb
from tqdm.auto import tqdm

# import engine module
from sirf.STIR import MessageRedirector
msg_red = MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')
import sirf.STIR as spect
cd_to_working_dir('SPECT', 'SPECT_acquisition_model')

In [ ]:
def create_sample_image(image):
    '''fill the image with some simple geometric shapes.'''
    image.fill(0)
    # create a shape
    shape = spect.EllipticCylinder()
    shape.set_length(400)
    shape.set_radii((100, 40))
    shape.set_origin((0, 60, 10))

    # add the shape to the image
    image.add_shape(shape, scale = 1)

    # add another shape
    shape.set_radii((30, 30))
    shape.set_origin((60, -30, 10))
    image.add_shape(shape, scale = 1.5)

    # add another shape
    shape.set_origin((-60, -30, 10))
    image.add_shape(shape, scale = 0.75)


In [ ]:
templ_sino = spect.AcquisitionData(os.path.join(examples_data_path('SPECT'), '','template_sinogram.hs'))

In [ ]:
# create image with suitable sizes
image = templ_sino.create_uniform_image()
create_sample_image(image)
image.write("simulated_image.hv")
# z-pixel coordinate of the xy-cross-section to show
z = image.dimensions()[0]//2

In [ ]:
# show the phantom image
image_array = image.as_array()
show_2D_array('Phantom image', image_array[z,:,:])

In [ ]:
# select acquisition model that implements the geometric
# forward projection by a ray tracing matrix multiplication
acq_model_matrix = spect.SPECTUBMatrix();
acq_model = spect.AcquisitionModelUsingMatrix(acq_model_matrix)

In [ ]:
# require same number slices and equal z-sampling for projection data & image
image = image.zoom_image(zooms=(0.5, 1.0, 1.0))
print('projecting image...')

In [ ]:
# project the image to obtain simulated acquisition data
    # data from raw_data_file is used as a template
acq_model.set_up(templ_sino, image)
simulated_data = templ_sino.get_uniform_copy()
acq_model.forward(image, 0, 1, simulated_data)

In [ ]:
# show simulated acquisition data
simulated_data_as_array = simulated_data.as_array()
middle_slice=simulated_data_as_array.shape[0]//2
show_2D_array('Forward projection', simulated_data_as_array[0, middle_slice,:,:])
# write data
simulated_data.write("simulated_data.hs")

In [ ]:
print('backprojecting the forward projection...')
# backproject the computed forward projection
back_projected_image = acq_model.backward(simulated_data, 0, 1)

back_projected_image_as_array = back_projected_image.as_array()
show_2D_array('Backprojection', back_projected_image_as_array[z,:,:])
